在上一章节，我们对赛题的数据进行了读取，并在末尾给出了两个小作业。如果你顺利完成了作业，那么你基本上对`Python`也比较熟悉了。在本章我们将使用传统机器学习算法来完成新闻分类的过程，将会结束到赛题的核心知识点。

## **Task3 基于机器学习的文本分类**

在本章我们将开始使用机器学习模型来解决文本分类。机器学习发展比较广，且包括多个分支，本章侧重使用传统机器学习，从下一章开始是基于深度学习的文本分类。

### **学习目标**

- 学会TF-IDF的原理和使用
- 使用sklearn的机器学习模型完成文本分类

### **机器学习模型**

机器学习是对能通过经验自动改进的计算机算法的研究。机器学习通过历史数据**训练**出**模型**对应于人类对经验进行**归纳**的过程，机器学习利用**模型**对新数据进行**预测**对应于人类利用总结的**规律**对新问题进行**预测**的过程。


机器学习有很多种分支，对于学习者来说应该优先掌握机器学习算法的分类，然后再其中一种机器学习算法进行学习。由于机器学习算法的分支和细节实在是太多，所以如果你一开始就被细节迷住了眼，你就很难知道全局是什么情况的。


如果你是机器学习初学者，你应该知道如下的事情：

1. 机器学习能解决一定的问题，但不能奢求机器学习是万能的；
2. 机器学习算法有很多种，看具体问题需要什么，再来进行选择；
3. 每种机器学习算法有一定的偏好，需要具体问题具体分析；


<div align=center><img src="http://jupter-oss.oss-cn-hangzhou.aliyuncs.com/public/files/image/1095279501877/1594909490018_ZFUWhFUSbB.jpg" width="50%" height="50%"></div>

### **文本表示方法 Part1**

在机器学习算法的训练过程中，假设给定$N$个样本，每个样本有$M$个特征，这样组成了$N×M$的样本矩阵，然后完成算法的训练和预测。同样的在计算机视觉中可以将图片的像素看作特征，每张图片看作hight×width×3的特征图，一个三维的矩阵来进入计算机进行计算。

但是在自然语言领域，上述方法却不可行：文本是不定长度的。文本表示成计算机能够运算的数字或向量的方法一般称为词嵌入（Word Embedding）方法。词嵌入将不定长的文本转换到定长的空间内，是文本分类的第一步。

#### **One-hot**

这里的One-hot与数据挖掘任务中的操作是一致的，即将每一个单词使用一个离散的向量表示。具体将每个字/词编码一个索引，然后根据索引进行赋值。

One-hot表示方法的例子如下：

```python
句子1：我 爱 北 京 天 安 门
句子2：我 喜 欢 上 海
```

首先对所有句子的字进行索引，即将每个字确定一个编号：

```python
{
	'我': 1, '爱': 2, '北': 3, '京': 4, '天': 5,
  '安': 6, '门': 7, '喜': 8, '欢': 9, '上': 10, '海': 11
}
```

在这里共包括11个字，因此每个字可以转换为一个11维度稀疏向量：

```
我：[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
爱：[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
...
海：[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1]
```

#### **Bag of Words**

Bag of Words（词袋表示），也称为Count Vectors，每个文档的字/词可以使用其出现次数来进行表示。

```python
句子1：我 爱 北 京 天 安 门
句子2：我 喜 欢 上 海
```

直接统计每个字出现的次数，并进行赋值：

```python
句子1：我 爱 北 京 天 安 门
转换为 [1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]

句子2：我 喜 欢 上 海
转换为 [1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1]
```

在sklearn中可以直接`CountVectorizer`来实现这一步骤：

```
from sklearn.feature_extraction.text import CountVectorizer
corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]
vectorizer = CountVectorizer()
vectorizer.fit_transform(corpus).toarray()
```

#### **N-gram**

N-gram与Count Vectors类似，不过加入了相邻单词组合成为新的单词，并进行计数。

如果N取值为2，则句子1和句子2就变为：

```
句子1：我爱 爱北 北京 京天 天安 安门
句子2：我喜 喜欢 欢上 上海
```

#### **TF-IDF**

TF-IDF 分数由两部分组成：第一部分是**词语频率**（Term Frequency），第二部分是**逆文档频率**（Inverse Document Frequency）。其中计算语料库中文档总数除以含有该词语的文档数量，然后再取对数就是逆文档频率。

```
TF(t)= 该词语在当前文档出现的次数 / 当前文档中词语的总数
IDF(t)= log_e（文档总数 / 出现该词语的文档总数）
```

### **基于机器学习的文本分类**

接下来我们将对比不同文本表示算法的精度，通过本地构建验证集计算F1得分。

In [1]:
# Count Vectors + RidgeClassifier

import pandas as pd

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import f1_score

train_df = pd.read_csv('../data/train_set.csv', sep='\t', nrows=15000)

vectorizer = CountVectorizer(max_features=3000)
train_test = vectorizer.fit_transform(train_df['text'])

clf = RidgeClassifier()
clf.fit(train_test[:10000], train_df['label'].values[:10000])

val_pred = clf.predict(train_test[10000:])
print(f1_score(train_df['label'].values[10000:], val_pred, average='macro'))
# 0.74

0.7406241569237678


In [2]:
# TF-IDF +  RidgeClassifier

import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import f1_score

train_df = pd.read_csv('../data/train_set.csv', sep='\t', nrows=15000)

tfidf = TfidfVectorizer(ngram_range=(1,3), max_features=3000)
train_test = tfidf.fit_transform(train_df['text'])

clf = RidgeClassifier()
clf.fit(train_test[:10000], train_df['label'].values[:10000])

val_pred = clf.predict(train_test[10000:])
print(f1_score(train_df['label'].values[10000:], val_pred, average='macro'))
# 0.87

0.8721598830546126


### **本章小结**

本章我们介绍了基于机器学习的文本分类方法，并完成了两种方法的对比。

### **本章作业**

1. 尝试改变TF-IDF的参数，并验证精度
2. 尝试使用其他机器学习模型，完成训练和验证

In [3]:
# TF-IDF +  LinearSVC

import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.linear_model import RidgeClassifier
from sklearn.metrics import f1_score

train_df = pd.read_csv('../data/train_set.csv', sep='\t', nrows=15000)

tfidf = TfidfVectorizer(ngram_range=(1, 3),min_df=6, max_df=0.9,use_idf=1,smooth_idf=1, sublinear_tf=1)
train_test = tfidf.fit_transform(train_df['text'])

clf = LinearSVC()
clf.fit(train_test[:10000], train_df['label'].values[:10000])

val_pred = clf.predict(train_test[10000:])
print(f1_score(train_df['label'].values[10000:], val_pred, average='macro'))

0.9074811239786598


In [15]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer
from sklearn.svm import LinearSVC
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.metrics import f1_score
# import dinglingling
# from dinglingling import wx_reminder
import datetime


# SCKEY = "SCU107128T841830fb3892a424180b3c318258798a5f1c42014351f"

In [21]:
# @wx_reminder(SCKEY=SCKEY, remind_started=False)
def train():
    print(datetime.datetime.now().strftime('start: %Y-%m-%d %H:%M:%S')) #当前时间
    train_df = pd.read_csv('../data/train_set.csv', sep='\t', nrows=15000)
    df_train_x = train_df[:10000]['text']
    df_train_y = train_df['label'].values[:10000]
    df_test = pd.read_csv('../data/test_a.csv', sep='\t')

    
    tfidf = TfidfVectorizer(ngram_range=(1,2))
    # train_test = tfidf.fit_transform(train_df['text'])
    classifier = LinearSVC()
    pipeline = Pipeline([('tfidf', tfidf),('clf', classifier)])

    parameters = {'tfidf__ngram_range': ((1, 2), (1, 3)),
                  'tfidf__min_df': (3, 6, 8),
                  'tfidf__max_df':(0.7, 0.9, 1.6),
                  'clf__C': (1.0, 2.0, 3.0)}

    skf = StratifiedKFold(n_splits=5, random_state=1)
    gs = GridSearchCV(estimator=pipeline, param_grid=parameters,
                      scoring='f1_macro', cv=skf, verbose=3)
    gs.fit(df_train_x, train_df['label'].values[:10000])

    print("Best score: %0.3f" % gs.best_score_)
    best_parameters = gs.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))

    x_test = gs.best_estimator_.named_steps['tfidf'].transform(df_test)
    y_test = gs.best_estimator_.named_steps['clf'].predict(x_test)
#     df_test['label'] = y_test.tolist()

    test=pd.DataFrame(data=y_test)
    print(test)
    test.to_csv('./result.csv', index=0, header=0)
    print(datetime.datetime.now().strftime('end: %Y-%m-%d %H:%M:%S')) #当前时间

In [22]:
train()

start: 2020-07-26 15:27:17
Fitting 5 folds for each of 54 candidates, totalling 270 fits
[CV] clf__C=1.0, tfidf__max_df=0.7, tfidf__min_df=3, tfidf__ngram_range=(1, 2) 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV]  clf__C=1.0, tfidf__max_df=0.7, tfidf__min_df=3, tfidf__ngram_range=(1, 2), score=0.895, total=  37.5s
[CV] clf__C=1.0, tfidf__max_df=0.7, tfidf__min_df=3, tfidf__ngram_range=(1, 2) 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   37.4s remaining:    0.0s


[CV]  clf__C=1.0, tfidf__max_df=0.7, tfidf__min_df=3, tfidf__ngram_range=(1, 2), score=0.895, total=  26.8s
[CV] clf__C=1.0, tfidf__max_df=0.7, tfidf__min_df=3, tfidf__ngram_range=(1, 2) 


...

[CV]  clf__C=2.0, tfidf__max_df=1.6, tfidf__min_df=8, tfidf__ngram_range=(1, 3), score=0.908, total= 1.1min
[CV] clf__C=2.0, tfidf__max_df=1.6, tfidf__min_df=8, tfidf__ngram_range=(1, 3) 
[CV]  clf__C=2.0, tfidf__max_df=1.6, tfidf__min_df=8, tfidf__ngram_range=(1, 3), score=0.882, total= 1.0min
[CV] clf__C=3.0, tfidf__max_df=0.7, tfidf__min_df=3, tfidf__ngram_range=(1, 2) 
[CV]  clf__C=3.0, tfidf__max_df=0.7, tfidf__min_df=3, tfidf__ngram_range=(1, 2), score=0.898, total=  30.9s
[CV] clf__C=3.0, tfidf__max_df=0.7, tfidf__min_df=3, tfidf__ngram_range=(1, 2) 
[CV]  clf__C=3.0, tfidf__max_df=0.7, tfidf__min_df=3, tfidf__ngram_range=(1, 2), score=0.897, total=  30.4s
[CV] clf__C=3.0, tfidf__max_df=0.7, tfidf__min_df=3, tfidf__ngram_range=(1, 2) 
[CV]  clf__C=3.0, tfidf__max_df=0.7, tfidf__min_df=3, tfidf__ngram_range=(1, 2), score=0.891, total=  30.2s
[CV] clf__C=3.0, tfidf__max_df=0.7, tfidf__min_df=3, tfidf__ngram_range=(1, 2) 
[CV]  clf__C=3.0, tfidf__max_df=0.7, tfidf__min_df=3, tfidf_

[CV]  clf__C=3.0, tfidf__max_df=0.9, tfidf__min_df=6, tfidf__ngram_range=(1, 2), score=0.892, total=  28.9s
[CV] clf__C=3.0, tfidf__max_df=0.9, tfidf__min_df=6, tfidf__ngram_range=(1, 2) 
[CV]  clf__C=3.0, tfidf__max_df=0.9, tfidf__min_df=6, tfidf__ngram_range=(1, 2), score=0.908, total=  28.5s
[CV] clf__C=3.0, tfidf__max_df=0.9, tfidf__min_df=6, tfidf__ngram_range=(1, 2) 
[CV]  clf__C=3.0, tfidf__max_df=0.9, tfidf__min_df=6, tfidf__ngram_range=(1, 2), score=0.878, total=  29.8s
[CV] clf__C=3.0, tfidf__max_df=0.9, tfidf__min_df=6, tfidf__ngram_range=(1, 3) 
[CV]  clf__C=3.0, tfidf__max_df=0.9, tfidf__min_df=6, tfidf__ngram_range=(1, 3), score=0.901, total= 1.1min
[CV] clf__C=3.0, tfidf__max_df=0.9, tfidf__min_df=6, tfidf__ngram_range=(1, 3) 
[CV]  clf__C=3.0, tfidf__max_df=0.9, tfidf__min_df=6, tfidf__ngram_range=(1, 3), score=0.898, total= 1.1min
[CV] clf__C=3.0, tfidf__max_df=0.9, tfidf__min_df=6, tfidf__ngram_range=(1, 3) 
[CV]  clf__C=3.0, tfidf__max_df=0.9, tfidf__min_df=6, tfidf_

[CV]  clf__C=3.0, tfidf__max_df=1.6, tfidf__min_df=8, tfidf__ngram_range=(1, 3), score=0.899, total=  56.2s
[CV] clf__C=3.0, tfidf__max_df=1.6, tfidf__min_df=8, tfidf__ngram_range=(1, 3) 
[CV]  clf__C=3.0, tfidf__max_df=1.6, tfidf__min_df=8, tfidf__ngram_range=(1, 3), score=0.902, total=  56.2s
[CV] clf__C=3.0, tfidf__max_df=1.6, tfidf__min_df=8, tfidf__ngram_range=(1, 3) 
[CV]  clf__C=3.0, tfidf__max_df=1.6, tfidf__min_df=8, tfidf__ngram_range=(1, 3), score=0.912, total=  55.4s
[CV] clf__C=3.0, tfidf__max_df=1.6, tfidf__min_df=8, tfidf__ngram_range=(1, 3) 
[CV]  clf__C=3.0, tfidf__max_df=1.6, tfidf__min_df=8, tfidf__ngram_range=(1, 3), score=0.882, total=  56.7s


[Parallel(n_jobs=1)]: Done 270 out of 270 | elapsed: 238.2min finished


Best score: 0.899
	clf__C: 3.0
	tfidf__max_df: 0.9
	tfidf__min_df: 6
	tfidf__ngram_range: (1, 3)
   0
0  8
end: 2020-07-26 19:26:35
